# The Pipeline 

## step1 : loading an embedding model

In [1]:
%%capture
from step1 import *
from ImportsAndDatasets import *

In [2]:
import pandas as pd

from parse_drugbank_data  import parse_drugbank ,convert_properties , extract_smiles

from preprocess_df_for_smiles import *

In [3]:
df = parse_drugbank("/home/u111169/wrkdir/mgh-project/datasets/drugbank_data/full_database.xml")

In [4]:
%%capture
approved_df, not_approved_df , lookup_table = preprocess_df_for_smiles(df)

In [5]:
df_properties = lookup_table[["name","description", "average_mass", "toxicity","groups", "labels" ,"approval_stat", "SMILES", "experimental_properties", "calculated_properties" ]]

In [6]:
train_df , val_df , test_df , dataset_train , dataset_df , dataset_test = train_valid_test_split(df_properties)

In [7]:
train_df = train_df[: 5]
val_df = val_df[: 5]
test_df = test_df[: 5]

In [28]:
def get_dict_of_approved_and_unapproved(df, dataset_train, dataset="val" , number_of_simmilar_in_each=5):
    """
    """
    df["Dicts"] = None
    for inst in range(len(df)): 
        app_dict = {}
        most_app , most_nonapp , tot  = get_most_app_and_most_nonapp(inst, df, dataset_train, dataset , number_of_simmilar_in_each)
        app_dict["most_app"]=most_app 
        app_dict["most_nonapp"]=most_nonapp 
        df["Dicts"][inst] = app_dict
    return df

In [18]:
def get_value(df, smiles_value, column_name):
    """
    Retrieves a value from a DataFrame based on a SMILES value and column name.

    Args:
        df (pd.DataFrame): The DataFrame to search in.
        smiles_value (str): The SMILES value to search for.
        column_name (str): The name of the column to retrieve the value from.

    Returns:
        The value from the specified column and SMILES value, or None if not found.
    """
    row = df[df["SMILES"] == smiles_value].reset_index()
    
    if not row.empty:
        return row[column_name][0]
    return None

In [19]:
def get_list_of_most(most_app:pd.DataFrame , df_properties): 
    final_list_of_most = []
    for i in most_app["SMILES"]: 
        dic = {}
        dic["name"] = get_value(df_properties,i, "name")
        dic["simmilarity score"] = most_app.loc[most_app["SMILES"] == i, "sim"].values
        dic["approval status"] = get_value(df_properties,i, "approval_stat")
        dic["average mass"] = get_value(df_properties , i, "average_mass")
        dic["toxicity"] = get_value(df_properties , i, "toxicity") 
        dic["descriptions"] = get_value(df_properties , i, "description") 
        final_list_of_most.append(dic) 
    return final_list_of_most
        

In [25]:
def final_dataset_containing_drug_information_as_list_ready_for_prompt(df, df_properties):
    # Initialize the new columns with None
    df["list_of_most_approved_info"] = None
    df["list_of_most_nonapproved_info"] = None
    
    # Iterate over each row in the DataFrame
    for i in range(len(df)):
        list_of_most_approved_info = get_list_of_most(df["Dicts"][i]["most_app"], df_properties)
        list_of_most_nonapproved_info = get_list_of_most(df["Dicts"][i]["most_nonapp"], df_properties)
        df.at[i, "list_of_most_approved_info_for_n_simmilar"] = list_of_most_approved_info
        df.at[i, "list_of_most_nonapproved_info_for_n_simmilar"] = list_of_most_nonapproved_info
        
    return df

In [ ]:
import time 
start = time.time()

a = get_dict_of_approved_and_unapproved(train_df, dataset_train, dataset="train" , number_of_simmilar_in_each=5)
final_df = final_dataset_containing_drug_information_as_list_ready_for_prompt(a , df_properties)

end=time.time()
e = start - end
print(e)

## prompt

In [36]:
sys_prompt = "you are an expert chemist "

In [40]:
input_prompt_1 = f""" {sys_prompt}
your task is to do reasoning based on the provided informaion that why the compound X is {approval_stat}
I trained a model that can predict the most simmilar molecules to  compound X. this model will give the five most simmilar and five most dissimillar moleculs based on the state of approvality. 
you know that the compound X is {approval_stat} and your task is to reason why this will be {approval_stat} based on the provided information. 
the most simmilar approved small molecules with their properties that my model predict to this coumpound are: 
{list_of_most_approved} 
and the most simmilar non approved small moleculs with their properties are: 
{list_of_most_nonapproved} 
"""

In [38]:
input_prompt_2 = f"""{sys_prompt}
Your task is to provide a detailed analysis explaining why compound X has an approval status of {final_df["approval_stat"][0]}.

I have developed a model that identifies molecules most similar to compound X. This model outputs two lists:
1. The five most similar approved small molecules (with their properties).
2. The five most similar non-approved small molecules (with their properties).

Using the provided data:
- Approved molecules: {final_df["list_of_most_approved_info"][0]}
- Non-approved molecules: {final_df["list_of_most_nonapproved_info"][0]}

Please analyze the data and reason which properties or patterns contribute to compound X being {final_df["approval_stat"][0]}. In your explanation, compare the characteristics observed in both lists to support your reasoning.
"""


In [43]:
input_prompt_3 = f"""
Your task is to analyze the likelihood of compound X receiving regulatory approval based on its similarity to known molecules.  

I have developed a model that identifies molecules most similar to compound X. This model outputs two lists:  
1. The five most similar approved small molecules (with their properties and similarity scores).  
2. The five most similar non-approved small molecules (with their properties and similarity scores).  

Using the provided data:  
- Approved molecules (with similarity scores): {list_of_most_approved}  
- Non-approved molecules (with similarity scores): {list_of_most_nonapproved}  

Please analyze the molecular properties, trends, and distinguishing features in both lists. Take the similarity scores into account when evaluating the degree of resemblance between compound X and the approved/non-approved molecules.  

Based on these comparisons, determine whether compound X is more likely to be approved or not. Justify your reasoning by highlighting key characteristics that align with approved or non-approved compounds, and discuss how the similarity scores influence your prediction.  

"""

In [44]:
input_prompt_3

"\nYour task is to analyze the likelihood of compound X receiving regulatory approval based on its similarity to known molecules.  \n\nI have developed a model that identifies molecules most similar to compound X. This model outputs two lists:  \n1. The five most similar approved small molecules (with their properties and similarity scores).  \n2. The five most similar non-approved small molecules (with their properties and similarity scores).  \n\nUsing the provided data:  \n- Approved molecules (with similarity scores): [{'name': 'Fluorodopa (18F)', 'simmilarity score': array([0.99665236]), 'approval status': 'approved', 'chemical properties': [{'logP': '-2'}, {'logS': '-2'}, {'Water Solubility': '2.39e+00 g/l'}, {'logP': '-1.6'}, {'IUPAC Name': '(2S)-2-amino-3-[2-(¹⁸F)fluoro-4,5-dihydroxyphenyl]propanoic acid'}, {'Traditional IUPAC Name': '6-(18F)Fluoro-L-DOPA'}, {'Molecular Weight': '214.183'}, {'Monoisotopic Weight': '214.061920473'}, {'SMILES': 'N[C@@H](CC1=CC(O)=C(O)C=C1[18F])C(